### Import libraries

In [1]:
import os
import copy
import torch
import pickle
import pprint
import random
import shutil
import random
import torch
import hashlib
import numpy as np
import pandas as pd
import torch.nn as nn
import tqdm.notebook as tqdm
from torch import optim
from sklearn.metrics import *
from sklearn.preprocessing import *
from collections import Counter
from collections import OrderedDict, defaultdict
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_sequence
from gensim.models.word2vec import Word2Vec
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from gensim.models.keyedvectors import KeyedVectors
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [2]:
!nvidia-smi

Mon Nov 28 13:58:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla M40 24GB      Off  | 00000000:04:00.0 Off |                    0 |
| N/A   42C    P8    16W / 250W |      3MiB / 22945MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla M40 24GB      Off  | 00000000:82:00.0 Off |                    0 |
| N/A   

In [27]:
device

device(type='cuda', index=2)

In [3]:
cache_dir =  "../cache"
data_dir = "../data/mimic3_new"
ID_COL_NAME = "HADM_ID"
LABEL_COL_NAME = "ICD9_CODE"
TEXT_COL_NAME = "TEXT"

## Constants

In [4]:
BATCH_SIZE = 8
n_epoch=10
optimiser="adamw" # "adam", "sgd", "adadelta", "adamw","adagrad"
main_metric="micro_f1"
multilabel=1
shuffle_data=1
dropout=0.3
max_seq_length=2500
min_seq_length=-1
min_word_frequency=-1
embedding_mode="word2vec"
embedding_size=100
embedding_file="../data/mimic3_new/processed_50.embed"
d_a=256 #"The dimension of the first dense layer for self attention"
n_labels=50

#RNN
hidden_size =  256
n_layers = 1
bidirectional =1
use_last_hidden_state=0

In [5]:
class Vocab(object):
    def __init__(self,
                 training_data: list,
                 training_labels: list,
                 min_word_frequency: int = -1,
                 max_vocab_size: int = -1,
                 word_embedding_mode: str = "word2vec",
                 word_embedding_file: str = None,
                 use_gpu: bool = True
                 ):
        self.device = torch.device("cuda" if torch.cuda.is_available() and use_gpu else "cpu")
        self.word_embedding_mode = word_embedding_mode
        self.word_embedding_file = word_embedding_file
        self.word_embedding_size = 100
        self.word_embeddings = None

        self.training_data = training_data

        self.PAD_TOKEN = '_PAD'
        self.UNK_TOKEN = '_UNK'
        self.word2index = None
        self.index2word = None

        self.label2index = None
        self.index2label = None

        self.vocab_words = [self.PAD_TOKEN, self.UNK_TOKEN]
        self.all_labels = []

        self.min_word_frequency = min_word_frequency
        self.max_vocab_size = max_vocab_size

        self.update_labels(training_labels)

    def index_of_word(self,word: str) -> int:
        try:
            return self.word2index[word]
        except:
            return self.word2index[self.UNK_TOKEN]

    def index_of_label(self,label: str) -> int:
        try:
            return self.label2index[label]
        except:
            return 0
    def update_labels(self, labels):
        self.all_labels = []
        self.index2label = []
        self.label2index = []
        all_labels = list(sorted(labels))
        self.label2index = {label: idx for idx, label in enumerate(all_labels)}
        self.index2label = {idx: label for idx, label in enumerate(all_labels)}
        self.all_labels = all_labels
            
    def prepare_vocab(self):
        self._build_vocab()

        # load pretrain word embeddings
        if self.word_embedding_file is not None:
            self.word_embeddings = torch.FloatTensor(self._load_embeddings())

    def _build_vocab(self):
        all_words_df = pd.read_csv('../data/mimic3_new/vocab.csv',header=None)
        all_words = all_words_df.iloc[:,0].tolist()
        all_words.sort()

        self.vocab_words += all_words

        self.word2index = {word: idx for idx, word in enumerate(self.vocab_words)}
        self.index2word = {idx: word for idx, word in enumerate(self.vocab_words)}


    def _load_embeddings(self):
        if self.word_embedding_file is None:
            return None
        return self._load_word_embeddings()

    def _load_word_embeddings(self):
        unknown_vec = np.random.uniform(-0.25, 0.25, self.word_embedding_size)
        embeddings = [unknown_vec] * (len(self.vocab_words))
        embeddings[0] = np.zeros(self.word_embedding_size)
        for line in open(self.word_embedding_file, "rt"):
            split = line.rstrip().split(" ")
            word = split[0]
            vector = np.array([float(num) for num in split[1:]]).astype(np.float32)
#             print(word,': ',vector)
            if len(vector) > 0:
                if word in self.word2index:
                    embeddings[self.word2index[word]] = vector
        embeddings = np.array(embeddings, dtype=np.float32)

        return embeddings
    def n_words(self):
        return len(self.vocab_words)

    def n_labels(self):
        return len(self.all_labels)

In [6]:
def read_data(file_path) -> list:
    
    data = pd.read_csv(file_path)

    id_data = data[ID_COL_NAME]
    text_data = data[TEXT_COL_NAME]

    hierarchical_label_data = []
    label_data = data[LABEL_COL_NAME].tolist()
    
    output = []
    for i in tqdm.tqdm(range(len(label_data)), desc="Reading data"):
        labels = label_data[i].split(';')
        print(labels)
        output.append((text_data[i], labels, id_data[i]))
    return output

In [7]:
def load_cached_data(file_path: str) -> tuple:
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        f.close()
    return data["vocab"], data["training_data"], data["valid_data"], data["test_data"]

In [8]:
def prepare_data():
    cache_folder = "{}/{}".format(cache_dir, "mimic3_single_50")
#     device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
    save_vocab_file_name = "{}.pkl".format("save_vocab")
    cached_file_name = os.path.join(cache_folder, save_vocab_file_name)
    if os.path.exists(cached_file_name):
        vocab, training_data, valid_data, test_data = load_cached_data(cached_file_name)
        data = training_data + valid_data + test_data
        labels = []
        for (feature, l, _) in data:
            labels.extend(l)
        unique_labels=list(set(labels))
        n_labels = len(unique_labels)
        labels = unique_labels
        vocab.update_labels(labels)
        print("Loaded vocab and data from file")
    else:
        training_data = read_data( data_dir + "/train.csv")
        valid_data = read_data( data_dir + "/dev.csv")
        test_data = read_data( data_dir + "/test.csv")
    #     print(len(training_data),len(valid_data),len(test_data))
        data = training_data + valid_data + test_data
        labels = []
        for (feature, l, _) in data:
            labels.extend(l)
        unique_labels=list(set(labels))
        n_labels = len(unique_labels)
        labels = unique_labels
        vocab = Vocab(data, labels,
                          min_word_frequency,
                          word_embedding_mode=embedding_mode,
                          word_embedding_file=embedding_file)
        print("Preparing the vocab")
        vocab.prepare_vocab()
        saved_objects = {"vocab": vocab,
                         "training_data": training_data,
                         "valid_data": valid_data,
                         "test_data": test_data}
        with open(cached_file_name, 'wb') as f:
            pickle.dump(saved_objects, f, pickle.HIGHEST_PROTOCOL)
            f.close()
        print("Saved vocab and data to files")
    return training_data, valid_data, test_data, vocab, cached_file_name

In [9]:
training_data, valid_data, test_data, vocab, saved_vocab_path = prepare_data()

Loaded vocab and data from file


In [14]:
class EmbeddingLayer(nn.Module):
    def __init__(self,
                 embedding_mode: str,
                 pretrained_word_embeddings: torch.Tensor,
                 vocab_size: int,
                 embedding_size: int):
        
        self.embedding_mode = embedding_mode
        super(EmbeddingLayer, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.embeddings.weight = nn.Parameter(copy.deepcopy(pretrained_word_embeddings), requires_grad=False)
        self.output_size = embedding_size
        
    def forward(self, batch_data: torch.LongTensor): # batch_data shape: [batch_size x max_padded_text_length]
        embeds = self.embeddings(batch_data)  # [batch_size x max_seq_size x embedding_size]
        return embeds

In [15]:
class LinearLayer(nn.Module):
    def __init__(self, size: int, n_labels=None):
        super(LinearLayer, self).__init__()
        self.size = size
        self.d_a = d_a
        self.n_labels = n_labels
        self.linear = nn.Linear(self.size, self.n_labels, bias = True)
        self._init_weights(mean=0.0, std=0.03)

    def _init_weights(self, mean=0.0, std=0.03) -> None:
        torch.nn.init.normal_(self.linear.weight,mean, std)
        self.linear.bias.data.fill_(0)

    def forward(self, x):
        weighted_output = self.linear(x)   
        return weighted_output

In [16]:
class Model(nn.Module):
    def __init__(self, vocab: Vocab):
        super(Model, self).__init__()
        self.vocab_size = vocab.n_words()
        self.vocab = vocab
        self.use_last_hidden_state = use_last_hidden_state
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.bidirectional = bool(bidirectional)
        self.n_directions = int(self.bidirectional) + 1
        self.output_size = self.hidden_size * self.n_directions

        self.dropout = dropout
        self.embedding = EmbeddingLayer(embedding_mode=embedding_mode,
                                     embedding_size=embedding_size,
                                     pretrained_word_embeddings=vocab.word_embeddings,
                                     vocab_size=vocab.n_words())
        
        self.rnn = nn.LSTM(self.embedding.output_size, self.hidden_size, num_layers=self.n_layers,
                               bidirectional=self.bidirectional, dropout=(self.dropout if self.n_layers > 1 else 0))
        
        self.use_dropout = dropout > 0
        self.dropout = nn.Dropout(dropout)
        
        self.linear = LinearLayer(size=self.output_size, n_labels=self.vocab.n_labels())

    def init_hidden(self, batch_size) -> Variable:
        h = Variable(torch.zeros(self.n_layers * self.n_directions, batch_size, self.hidden_size)).to(device)
        c = Variable(torch.zeros(self.n_layers * self.n_directions, batch_size, self.hidden_size)).to(device)
        return h, c

    def forward(self, batch_data: torch.LongTensor, lengths: torch.LongTensor) -> tuple:
        batch_size = batch_data.size()[0]
        hidden = self.init_hidden(batch_size)
        embeds = self.embedding(batch_data)
        
        if self.use_dropout:
            embeds = self.dropout(embeds)
            
        self.rnn.flatten_parameters()
        embeds = pack_padded_sequence(embeds, lengths.cpu(), batch_first=True)
        rnn_output, hidden = self.rnn(embeds, hidden) 
        hidden = hidden[0]
        rnn_output = pad_packed_sequence(rnn_output)[0]
        rnn_output = rnn_output.permute(1, 0, 2)
        
        hidden_forward = hidden[-1]
        hidden_backward = hidden[0]        
        last_rnn_output = torch.cat((hidden_forward, hidden_backward), 1)
        
        output = self.linear(last_rnn_output)
        return output

In [17]:
model = Model(vocab)
model

Model(
  (embedding): EmbeddingLayer(
    (embeddings): Embedding(66322, 100)
  )
  (rnn): LSTM(100, 256, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (linear): LinearLayer(
    (linear): Linear(in_features=512, out_features=50, bias=True)
  )
)

In [18]:
class TextDataset(Dataset):
    def __init__(self, text_data: list, vocab, sort: bool = True):
        super(TextDataset, self).__init__()
        self.vocab = vocab
        self.multilabel = multilabel
        self.max_seq_length = max_seq_length
        self.min_seq_length = min_seq_length
        self.PAD_ID = self.vocab.index_of_word(self.vocab.PAD_TOKEN) # 0
        indexed_data = []
        self.n_instances = len(text_data)
        print('Length of text data:- ', self.n_instances)
        self.n_total_tokens = 0

        n_label_level = 1
        self.label_count = dict()
        self.labels = set()
        for text, labels, _id in tqdm.tqdm(text_data, unit="samples", desc="Processing data"):
            label_list = []
            for label in labels:
                if label in self.vocab.label2index:
                    label = self.vocab.index_of_label(label)
                    if label not in self.label_count:
                        self.label_count[label] = 1
                    else:
                        self.label_count[label] += 1
                    self.labels.add(label)
                    label_list.append(label)
                else:
                    continue

            if len(label_list) == 0:
                continue

            word_seq = []
            sent_words = text.strip().split()
#             print('sent words:- ', len(sent_words))
            for word in sent_words:
                word_idx = vocab.index_of_word(word)
#                 if word_idx==vocab.word2index['_UNK']:
#                     continue
                word_seq.append(word_idx)
                self.n_total_tokens += 1
                if len(word_seq) >= self.max_seq_length:
                    break
            if len(word_seq) > 0:
                indexed_data.append((word_seq, label_list, _id))
#             print('word sequence:- ', len(word_seq))

        if sort:
            self.indexed_data = sorted(indexed_data, key=lambda x: -len(x[0]))
        else:
            self.indexed_data = indexed_data
            self.shuffle_data()

        self.labels = sorted(list(self.labels))
        self.size = len(self.indexed_data)

    def shuffle_data(self):
        random.shuffle(self.indexed_data)

    def __getitem__(self, index):
        word_seq, label_list, _id = self.indexed_data[index]
        if len(word_seq) > self.max_seq_length:
            word_seq = word_seq[:self.max_seq_length]

        one_hot_label_list = [0] * self.vocab.n_labels()
        for label in label_list:
            one_hot_label_list[label] = 1
        return word_seq, one_hot_label_list, _id

    def __len__(self):
        return len(self.indexed_data)

In [19]:
def _load_and_cache_data(train_data, valid_data, test_data, vocab, saved_data_file_path):
    if os.path.exists(saved_data_file_path):
        try:
            with open(saved_data_file_path, 'rb') as f:
                data = pickle.load(f)
                data["train"].multilabel = bool(multilabel)
                data["valid"].multilabel = bool(multilabel)
                data["test"].multilabel = bool(multilabel)

                return data["train"], data["valid"], data["test"]
        except:
            pass

    # Build train/valid/test data loaders
    train_dataset = TextDataset(train_data, vocab,sort=True)

    valid_dataset = TextDataset(valid_data, vocab,sort=True)

    test_dataset = TextDataset(test_data, vocab, sort=True)
    # try:
    with open(saved_data_file_path, 'wb') as f:
        data = {"train": train_dataset, "valid": valid_dataset, "test": test_dataset}
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
    
    return train_dataset, valid_dataset, test_dataset

In [20]:
class TextDataLoader(DataLoader):
    def __init__(self, vocab, **kwargs):
        super(TextDataLoader, self).__init__( **kwargs)
        self.collate_fn = self._collate_fn # recieves is a list of tuples of (word seq, one_hot_label_list, _id)
        self.PAD_ID = vocab.index_of_word(vocab.PAD_TOKEN)
        self.vocab = vocab

    def _collate_fn(self, batch):
#         print('collate_fn called:- ')
#         print(len(batch[0][0]))
        length_batch = []

#         self.first_linear = nn.Linear(self.size, self.d_a, bias=False)
#         self.second_linear = nn.Linear(self.d_a, n_labels, bias=False)
#         self.third_linear = nn.Linear(self.size , n_labels, bias=True)
        feature_batch = []
        label_batch = []

        id_batch = []
        multilabel = True
        for features, labels, _id in batch: # labels are one-hot encoded
            feature_length = len(features)
            feature_batch.append(torch.LongTensor(features))

            length_batch.append(feature_length)
            label_batch.append(labels)
            id_batch.append(_id)

        feature_batch, label_batch, length_batch, id_batch = \
            self.sort_batch(feature_batch, label_batch, length_batch, id_batch)
        
        padded_batch = pad_sequence(feature_batch, batch_first=True)
        feature_batch = torch.LongTensor(padded_batch)
        
        label_batch = np.stack(label_batch, axis=0)
        
        label_batch = torch.FloatTensor(label_batch.tolist()) # converts list to tensor
#         print('before longtensor length batch',length_batch)
        length_batch = torch.LongTensor(length_batch)
#         print('after longtensor length batch',length_batch)

#         print('shape of feature batch:-', feature_batch.shape)
#         print('length batch:-', length_batch)
#         print('shape of label batch:-', label_batch.shape)
#         print('id batch:-', id_batch)
#         print('type of length batch:- ', type(length_batch))
        return feature_batch, label_batch, length_batch, id_batch

    @staticmethod
    def sort_batch(features, labels, lengths, id_batch):
        sorted_indices = sorted(range(len(features)), key=lambda i: features[i].size(0), reverse=True)
        sorted_features = []
        sorted_labels = []
        sorted_lengths = []
        sorted_ids = []

        for index in sorted_indices:
            sorted_features.append(features[index])

            sorted_labels.append(labels[index])
            sorted_lengths.append(lengths[index])

            sorted_ids.append(id_batch[index])

        return sorted_features, sorted_labels, sorted_lengths, sorted_ids

In [21]:
class Evaluator:
    def __init__(self,model: torch.nn.Module, vocab, criterions, n_training_labels):
        self.model = model
        self.vocab = vocab
        self.index_to_label = vocab.index2label
        self.multilabel = multilabel
        self.criterions = criterions
        self.n_training_labels = n_training_labels

    def evaluate(self, dataloader: TextDataLoader) -> dict:

        self.model.eval()
        pred_probs = []
        true_labels = []
        ids = []

        losses = []
        all_loss_list = []

        for text_batch, label_batch, length_batch, id_batch in tqdm.tqdm(dataloader, unit="batches", desc="Evaluating"):

            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device)
            length_batch = length_batch.to(device)
            
            true_labels.extend(label_batch.cpu().numpy())
            ids.extend(id_batch)
            
            with torch.no_grad():
                output = self.model(text_batch, length_batch)
#                 output, attn_weights = self.model(text_batch, length_batch)

            probs = [None] * len(output)
            
            loss = self.criterions(output, label_batch)
            output = torch.sigmoid(output)
            all_loss_list.append(loss.item())
            output = output.detach().cpu().numpy()
            probs = output.tolist()
            pred_probs.extend(output)
            losses.append(loss.item())

        scores = OrderedDict()
        scores = calculate_eval_metrics(ids, true_labels, pred_probs, self.multilabel)
        scores["loss"] = np.mean(all_loss_list).item()

        return scores

In [22]:
def calculate_eval_metrics(ids, true_labels, pred_probs, is_multilabel):
    true_labels = np.asarray(true_labels)
    pred_probs = np.asarray(pred_probs)
    pred_labels = np.rint(pred_probs)
    
    macro_scores = calculate_scores(true_labels, pred_labels, pred_probs, "macro", is_multilabel)
    micro_scores = calculate_scores(true_labels, pred_labels, pred_probs, "micro", is_multilabel)
    
    scores = macro_scores
    scores.update(micro_scores)
    scores["ids"] = ids
    scores["true_labels"] = true_labels
    scores["pred_probs"] = pred_probs
    return scores

def calculate_scores(true_labels, pred_labels, pred_probs, average="macro", is_multilabel=True):
    max_size = min(len(true_labels), len(pred_labels))
    true_labels = true_labels[: max_size]
    pred_labels = pred_labels[: max_size]
    pred_probs = pred_probs[: max_size]
    p_1 = 0
    p_5 = 0
    p_8 = 0
    p_10 = 0
    p_15 = 0
    if pred_probs is not None:
        if average == "macro":
            accuracy = macro_accuracy(true_labels, pred_labels)  # categorical accuracy
            precision, recall, f1 = macro_f1(true_labels, pred_labels)
            p_ks = precision_at_k(true_labels, pred_probs, [1, 5, 8, 10, 15])
            p_1 = p_ks[0]
            p_5 = p_ks[1]
            p_8 = p_ks[2]
            p_10 = p_ks[3]
            p_15 = p_ks[4]

        else:
            accuracy = micro_accuracy(true_labels, pred_labels)
            precision, recall, f1 = micro_f1(true_labels, pred_labels)
        auc_score = roc_auc(true_labels, pred_probs, average)
    else:
        auc_score = -1

    output = {"{}_precision".format(average): precision, "{}_recall".format(average): recall,
              "{}_f1".format(average): f1, "{}_accuracy".format(average): accuracy,
              "{}_auc".format(average): auc_score, "{}_P@1".format(average): p_1, "{}_P@5".format(average): p_5,
              "{}_P@8".format(average): p_8, "{}_P@10".format(average): p_10, "{}_P@15".format(average): p_15}
    
    return output

def average_scores(scores):
    avg_scores = dict()
    for key in scores:
        if key.startswith("level"):
            for metric in scores[key]:
                if metric.startswith("macro") or metric.startswith("micro") or metric == "loss":
                    if metric not in avg_scores:
                        avg_scores[metric] = []
                    avg_scores[metric].append(scores[key][metric])

    for metric in avg_scores:
        avg_scores[metric] = sum(avg_scores[metric]) / len(avg_scores[metric])
    return avg_scores

def union_size(x, y, axis):
    return np.logical_or(x, y).sum(axis=axis).astype(float)


def intersect_size(x, y, axis):
    return np.logical_and(x, y).sum(axis=axis).astype(float)

def macro_precision(true_labels, pred_labels):
    num = intersect_size(true_labels, pred_labels, 0) / (pred_labels.sum(axis=0) + 1e-10)
    return np.mean(num)


def macro_recall(true_labels, pred_labels):
    num = intersect_size(true_labels, pred_labels, 0) / (true_labels.sum(axis=0) + 1e-10)
#     print('macro recall num:- ', num)
    return np.mean(num)


def macro_f1(true_labels, pred_labels):
    prec = macro_precision(true_labels, pred_labels)
    rec = macro_recall(true_labels, pred_labels)
    if prec + rec == 0:
        f1 = 0.
    else:
        f1 = 2 * (prec * rec) / (prec + rec)
    return prec, rec, f1


def macro_accuracy(true_labels, pred_labels):
    num = intersect_size(true_labels, pred_labels, 0) / (union_size(true_labels, pred_labels, 0) + 1e-10)
    return np.mean(num)


def micro_precision(true_labels, pred_labels):
    flat_true = true_labels.ravel()
    flat_pred = pred_labels.ravel()
    if flat_pred.sum(axis=0) == 0:
        return 0.0
    return intersect_size(flat_true, flat_pred, 0) / flat_pred.sum(axis=0)


def micro_recall(true_labels, pred_labels):
    flat_true = true_labels.ravel()
    flat_pred = pred_labels.ravel()
    return intersect_size(flat_true, flat_pred, 0) / flat_true.sum(axis=0)

def micro_f1(true_labels, pred_labels):
    prec = micro_precision(true_labels, pred_labels)
    rec = micro_recall(true_labels, pred_labels)
    if prec + rec == 0:
        f1 = 0.
    else:
        f1 = 2 * (prec * rec) / (prec + rec)
    return prec, rec, f1


def micro_accuracy(true_labels, pred_labels):
    flat_true = true_labels.ravel()
    flat_pred = pred_labels.ravel()
    return intersect_size(flat_true, flat_pred, 0) / union_size(flat_true, flat_pred, 0)


def recall_at_k(true_labels, pred_probs, k):
    # num true labels in top k predictions / num true labels
    sortd = np.argsort(pred_probs)[:, ::-1]
    topk = sortd[:, :k]

    # get recall at k for each example
    vals = []
    for i, tk in enumerate(topk):
        num_true_in_top_k = true_labels[i, tk].sum()
        denom = true_labels[i, :].sum()
        vals.append(num_true_in_top_k / float(denom))

    vals = np.array(vals)
    vals[np.isnan(vals)] = 0.

    return np.mean(vals)


def precision_at_k(true_labels, pred_probs, ks=[1, 5, 8, 10, 15]):
    # num true labels in top k predictions / k
    sorted_pred = np.argsort(pred_probs)[:, ::-1]
    output = []
    for k in ks:
        topk = sorted_pred[:, :k]

        # get precision at k for each example
        vals = []
        for i, tk in enumerate(topk):
            if len(tk) > 0:
                num_true_in_top_k = true_labels[i, tk].sum()
                denom = len(tk)
                vals.append(num_true_in_top_k / float(denom))

        output.append(np.mean(vals))
    return output


def roc_auc(true_labels, pred_probs, average="macro"):
    if pred_probs.shape[0] <= 1:
        return

    fpr = {}
    tpr = {}
    if average == "macro":
        # get AUC for each label individually
        relevant_labels = []
        auc_labels = {}
        for i in range(true_labels.shape[1]):
            # only if there are true positives for this label
            if true_labels[:, i].sum() > 0:
                fpr[i], tpr[i], _ = roc_curve(true_labels[:, i], pred_probs[:, i])
                if len(fpr[i]) > 1 and len(tpr[i]) > 1:
                    auc_score = auc(fpr[i], tpr[i])
                    if not np.isnan(auc_score):
                        auc_labels["auc_%d" % i] = auc_score
                        relevant_labels.append(i)

        # macro-AUC: just average the auc scores
        aucs = []
        for i in relevant_labels:
            aucs.append(auc_labels['auc_%d' % i])
        score = np.mean(aucs)
    else:
        # micro-AUC: just look at each individual prediction
        flat_pred = pred_probs.ravel()
        fpr["micro"], tpr["micro"], _ = roc_curve(true_labels.ravel(), flat_pred)
        score = auc(fpr["micro"], tpr["micro"])

    return score


def multiclass_roc_auc(true_labels, pred_labels, average="macro"):
    lb = LabelBinarizer()
    lb.fit(true_labels)
    true_labels = lb.transform(true_labels)
    pred_labels = lb.transform(pred_labels)
    return roc_auc_score(true_labels, pred_labels, average=average)

def average_scores(scores):
    avg_scores = dict()
    for key in scores:
        if key.startswith("level"):
            for metric in scores[key]:
                if metric.startswith("macro") or metric.startswith("micro") or metric == "loss":
                    if metric not in avg_scores:
                        avg_scores[metric] = []
                    avg_scores[metric].append(scores[key][metric])

    for metric in avg_scores:
        avg_scores[metric] = sum(avg_scores[metric]) / len(avg_scores[metric])
    return avg_scores

def normalise_labels(labels, n_label):
    norm_labels = []
    for label in labels:
        one_hot_vector_label = [0] * n_label
        one_hot_vector_label[label] = 1
        norm_labels.append(one_hot_vector_label)
    return np.asarray(norm_labels)

In [23]:
class Trainer:
    def __init__(self, model: nn.Module,
                 train_dataloader: TextDataLoader,
                 valid_dataloader: TextDataLoader,
                 test_dataloader: TextDataLoader,
                 criterions,
                 optimiser,
#                  lr_scheduler,
                 vocab,
                 checkpoint_path,
                 ):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.test_dataloader = test_dataloader
        self.criterions = criterions
        self.optimiser = optimiser
#         self.lr_scheduler = lr_scheduler
        self.vocab = vocab

        self.multilabel = multilabel
        self.checkpoint_path = checkpoint_path
        self.n_training_labels = n_labels
        self.main_metric = main_metric
        self.start_epoch = 0

    def train_single_epoch(self, index):
        self.model.train()
        self.train_dataloader.dataset.shuffle_data()
        losses = []
        true_labels = []
        pred_probs = []
        ids = []
        all_loss_list = []
        progress_bar = tqdm.tqdm(self.train_dataloader, unit="batches", desc="Training at epoch #{}".format(index))
        progress_bar.clear()
        self.optimiser.zero_grad()
        batch_id = 0

        for text_batch, label_batch, length_batch, id_batch in progress_bar:
            batch_id += 1
            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device)
            length_batch = length_batch.to(device)

#             output, attn_weights = self.model(text_batch, length_batch)
            
            output = self.model(text_batch, length_batch)
            true_labels.extend(label_batch.cpu().numpy())

            ids.extend(id_batch)

            loss = self.criterions(output, label_batch)
            output = torch.sigmoid(output)
            all_loss_list.append(loss.item())
            output = output.detach().cpu().numpy()
            pred_probs.extend(output)
            loss.backward()
            losses.append(loss.item())
            
#             print('pred probs:- ', output)
#             print('true labels:- ', label_batch)

            self.optimiser.step()
#             self.lr_scheduler.step()
            self.optimiser.zero_grad()
        
#         print('pred labels', pred_probs[0:3])

        scores = OrderedDict()
        scores = calculate_eval_metrics(ids, true_labels, pred_probs, self.multilabel)
        scores["loss"] = np.mean(all_loss_list).item()

        progress_bar.refresh(True)
        progress_bar.clear(True)
        progress_bar.close()
#         print('lr {}'.format(self.optimiser.param_groups[0]['lr']))
        print('loss:- ', scores['loss'], '  micro auc:- ', scores['micro_auc'], '  macro auc:- ', scores['macro_auc'])
        return scores

    @staticmethod
    def format_number(number):
        return abs(round(number, ndigits=ndigits))

    def train(self, n_epoch: int = 100):
        evaluator = Evaluator(self.model, self.vocab, self.criterions, self.n_training_labels)
        for e in range(self.start_epoch + 1, n_epoch + 1):
            train_scores = self.train_single_epoch(e)
            valid_scores = evaluator.evaluate(self.valid_dataloader)
            print('loss:- ', valid_scores['loss'], '  micro auc:- ', valid_scores['micro_auc'], '  macro auc:- ', valid_scores['macro_auc'])
        test_scores = evaluator.evaluate(self.test_dataloader)
        return self.model, test_scores

In [24]:
def _train_model(train_data, valid_data, test_data, vocab, saved_data_file_path=None,checkpoint_path=None):
    model = Model(vocab)
    model.to(device)

    train_dataset, valid_dataset, test_dataset = _load_and_cache_data(train_data, valid_data, test_data,
                                                                      vocab, saved_data_file_path)
    train_dataloader = TextDataLoader(dataset=train_dataset, vocab=vocab, batch_size=BATCH_SIZE)

    valid_dataloader = TextDataLoader(dataset=valid_dataset, vocab=vocab, batch_size=BATCH_SIZE)

    test_dataloader = TextDataLoader(dataset=test_dataset, vocab=vocab, batch_size=BATCH_SIZE)

    optimiser = torch.optim.Adam(model.parameters(), lr = 0.001)
    
#     lr_scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=50)
    
    criterions = nn.BCEWithLogitsLoss(pos_weight = torch.tensor(8))

    trainer = Trainer(model=model,
                      train_dataloader=train_dataloader,
                      valid_dataloader=valid_dataloader,
                      test_dataloader=test_dataloader,
                      criterions=criterions,
                      optimiser=optimiser,
#                       lr_scheduler = lr_scheduler,
                      vocab=vocab,
                      checkpoint_path=checkpoint_path)
    best_model, scores = trainer.train(n_epoch=n_epoch)

    evaluator = Evaluator(model=best_model,vocab=vocab,criterions=criterions,
                          n_training_labels=n_labels)

    del model, optimiser, evaluator, trainer, criterions
    return best_model, scores  # either on valid or test

In [25]:
def run_with_validation(training_data, valid_data, test_data, vocab, saved_data_file_path):
    best_model, scores = _train_model(
        train_data=training_data, valid_data=valid_data, test_data=test_data,
        vocab=vocab, saved_data_file_path=saved_data_file_path, checkpoint_path="../model/checkpoint.pkl")

    return best_model, scores

In [26]:
model, test_scores = run_with_validation(training_data, valid_data, test_data, vocab, saved_data_file_path="{}.data.pkl".format(saved_vocab_path.split(".pkl")[0]))

RuntimeError: CUDA error: invalid device ordinal

In [755]:
print('micro auc:- ', test_scores['micro_auc'])
print('macro auc:- ', test_scores['macro_auc'])
print('\nmacro F1:- ', test_scores['macro_f1'])
print('micro_F1:- ', test_scores['micro_f1'])
print('\nP@5:- ', test_scores['macro_P@5'])
print('P@8:- ', test_scores['macro_P@8'])
print('P@15:- ', test_scores['macro_P@15'])

micro auc:-  0.8621733895126819
macro auc:-  0.8143088123516844

macro F1:-  0.43694100355330756
micro_F1:-  0.4430170575692963

P@5:-  0.4875650665124349
P@8:-  0.4050751879699248
P@15:-  0.2980913823019087
